# WikiArt dataset cleaning, extending

WikiArt: all art pieces stored as a picture in Wikipedia (till a certain date).<br>
Original gathering and steps is in the *datasets/originals/wikiart_initial.ipynb* file. This file contains the updates since 2024.02.05.

## Update 2024.02.08-10: Gather more information on painters via the Wikidata API

We take the current list of painters:

In [1]:
import pandas as pd

artists_wikiart = pd.read_csv('https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/wikiart_artists.csv')

,artist,styles,movement,styles_extended,pictures_count,birth_place,birth_year
0,Ad Reinhardt,"Abstract Art, Abstract Expressionism, Color Fi...",Abstract Expressionism,"{Abstract Art:15},{Abstract Expressionism:5},{...",52,Buffalo,1913.0
1,Adnan Coker,"Abstract Art, Abstract Expressionism",Abstract Art,"{Abstract Art:25},{Abstract Expressionism:3}",28,NaN,NaN
2,Akkitham Narayanan,Abstract Art,Abstract Art,{Abstract Art:17},17,Kerala,1939.0
3,Alberto Magnelli,"Abstract Art, Art Nouveau (Modern), Cubism, Ex...",Abstract Art,"{Abstract Art:19},{Art Nouveau (Modern):2},{Cu...",35,Florence,1888.0
4,Alekos Kontopoulos,"Abstract Art, Cubism, Expressionism, Post-Impr...",Social Realism,"{Abstract Art:26},{Cubism:5},{Expressionism:10...",79,Lamia,1904.0
...,...,...,...,...,...,...,...
3198,Serhij Schyschko,Unknown,Academic Art,{Unknown:9},9,NaN,NaN
3199,Vudon Baklytsky,Unknown,Soviet Nonconformist Art,{Unknown:46},46,NaN,NaN
3200,Wolfgang Tillmans,Unknown,Contemporary,{Unknown:9},9,Remscheid,1968.0
3201,Wu Daozi,Unknown,Tang Dynasty (618–907),{Unknown:8},8,Chang'an,680.0


Example:

In [2]:
import httpimport

with httpimport.remote_repo('https://raw.githubusercontent.com/me9hanics/sparql-wikidata-data-collection/main/'):
    import functions as external_functions

In [3]:
external_functions.get_all_person_info("Rembrandt")

{'name': 'Rembrandt',
 'birth_place': 'Leiden',
 'birth_date': '1606-07-15T00:00:00Z',
 'death_date': '1669-10-04T00:00:00Z',
 'death_place': 'Amsterdam',
 'gender': 'male',
 'citizenship': 'Dutch Republic',
 'occupation': ['drawer',
  'printmaker',
  'etcher',
  'art collector',
  'collector',
  'painter'],
 'work_locations': [{'location': 'Amsterdam',
   'start_time': '1623-01-01T00:00:00Z',
   'end_time': '1625-01-01T00:00:00Z',
   'point_in_time': None},
  {'location': 'Amsterdam',
   'start_time': '1631-01-01T00:00:00Z',
   'end_time': '1669-01-01T00:00:00Z',
   'point_in_time': None},
  {'location': 'Leiden',
   'start_time': '1625-01-01T00:00:00Z',
   'end_time': '1631-01-01T00:00:00Z',
   'point_in_time': None},
  {'location': 'Leiden',
   'start_time': '1620-01-01T00:00:00Z',
   'end_time': '1624-01-01T00:00:00Z',
   'point_in_time': None}]}

In [4]:
df = artists_wikiart.copy()
df['death_year'] = None
df['death_place'] = None
df['gender'] = None
df['citizenship'] = None
df['occupations'] = None
df['locations'] = None
df['locations_with_years'] = None

for painter in artists_wikiart['artist']:
    if painter['gender'] == None: #We already processed this painter
        continue
    painter_json = external_functions.get_person_info_retry_after(painter, placeofbirth = False, dateofbirth = False)
    if painter_json:
        df.loc[df['artist'] == painter, 'death_year'] = external_functions.find_year(painter_json['death_date'])
        df.loc[df['artist'] == painter, 'death_place'] = painter_json['death_place']
        df.loc[df['artist'] == painter, 'gender'] = painter_json['gender']
        df.loc[df['artist'] == painter, 'citizenship'] = painter_json['citizenship']
        df.loc[df['artist'] == painter, 'occupations'] = ', '.join((painter_json['occupation']))
        df.loc[df['artist'] == painter, 'locations'] = external_functions.get_places_from_response(painter_json)
        df.loc[df['artist'] == painter, 'locations_with_years'] = external_functions.get_places_with_years_from_response(painter_json)


df
        

Error fetching data for Alphonse Allais, status code: 429.
Attempt 1 of 3.
Error fetching data for Balcomb Greene, status code: 429.
Attempt 1 of 3.
Error fetching data for Constantin Brâncuși, status code: 429.
Attempt 1 of 3.
Error fetching data for Eduardo Chillida, status code: 429.
Attempt 1 of 3.
Error fetching data for Fernando Lanhas, status code: 429.
Attempt 1 of 3.
Error fetching data for Gerardo Dottori, status code: 429.
Attempt 1 of 3.
Error fetching data for Hilma af Klint, status code: 429.
Attempt 1 of 3.
Error fetching data for Jean Arp, status code: 429.
Attempt 1 of 3.
Error fetching data for John Marin, status code: 429.
Attempt 1 of 3.
Error fetching data for Lajos Kassak, status code: 429.
Attempt 1 of 3.
Error fetching data for Marcel Janco, status code: 429.
Attempt 1 of 3.
Error fetching data for Natalia Goncharova, status code: 429.
Attempt 1 of 3.
Error fetching data for Princess Fahrelnissa Zeid, status code: 429.
Attempt 1 of 3.
Error fetching data for S. 

ValueError: Must have equal len keys and value when setting with an iterable

### --- PARTIAL SAVE --- (2024.02.10 3:30 AM UTC)

In [11]:
df

,artist,styles,movement,styles_extended,pictures_count,birth_place,birth_year,death_year,death_place,gender,citizenship,occupations,locations,locations_with_years
0,Ad Reinhardt,"Abstract Art, Abstract Expressionism, Color Fi...",Abstract Expressionism,"{Abstract Art:15},{Abstract Expressionism:5},{...",52,Buffalo,1913.0,1967,New York City,male,United States of America,"painter, university teacher, printmaker, colla...",['New York City'],[]
1,Adnan Coker,"Abstract Art, Abstract Expressionism",Abstract Art,"{Abstract Art:25},{Abstract Expressionism:3}",28,NaN,NaN,None,None,None,None,None,None,None
2,Akkitham Narayanan,Abstract Art,Abstract Art,{Abstract Art:17},17,Kerala,1939.0,None,None,None,None,None,None,None
3,Alberto Magnelli,"Abstract Art, Art Nouveau (Modern), Cubism, Ex...",Abstract Art,"{Abstract Art:19},{Art Nouveau (Modern):2},{Cu...",35,Florence,1888.0,1971,Meudon,male,Italy,"illustrator, painter","['Florence', 'Paris']",[]
4,Alekos Kontopoulos,"Abstract Art, Cubism, Expressionism, Post-Impr...",Social Realism,"{Abstract Art:26},{Cubism:5},{Expressionism:10...",79,Lamia,1904.0,1975,Athens,male,Greece,"writer, painter",[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3198,Serhij Schyschko,Unknown,Academic Art,{Unknown:9},9,NaN,NaN,None,None,None,None,None,None,None
3199,Vudon Baklytsky,Unknown,Soviet Nonconformist Art,{Unknown:46},46,NaN,NaN,None,None,None,None,None,None,None
3200,Wolfgang Tillmans,Unknown,Contemporary,{Unknown:9},9,Remscheid,1968.0,None,None,None,None,None,None,None
3201,Wu Daozi,Unknown,Tang Dynasty (618–907),{Unknown:8},8,Chang'an,680.0,None,None,None,None,None,None,None


In [12]:
artists_wikiart.to_csv('datasets/saves/wikiart_artists.csv', index=False)
df.to_csv('datasets/wikiart_artists.csv', index=False)